2D Example demonstrating a spill and well separated by a stream.

In [58]:
import sys
import math
from io import StringIO
import os
import shutil
import platform
import numpy as np
from subprocess import check_output
import flopy

print(np.__version__)

modelpth = os.path.join('Cara_model')
modelname = 'Spill'
mfexe = 'mfnwt'
mtexe = 'mt3dusgs'
if platform.system() == 'Windows':
    mfexe += '.exe'
    mtexe += '.exe'

# Instantiate MODFLOW model
mf = flopy.modflow.Modflow(modelname, version='mfnwt', exe_name=mfexe, 
                           model_ws=modelpth)

# Output Control: Create a flopy output control object
oc = flopy.modflow.ModflowOc(mf, stress_period_data={(0,0):['save head','save budget']})

# Newton-Rhapson Solver: Create a flopy nwt package object
headtol = 0.0001
fluxtol = 0.06
maxiterout = 200
thickfact = 1E-005
linmeth = 2
iprnwt = 1
ibotav = 0
nwt = flopy.modflow.ModflowNwt(mf, headtol=headtol, fluxtol=fluxtol, maxiterout=maxiterout,
                               thickfact=thickfact, linmeth=linmeth, iprnwt=iprnwt, ibotav=ibotav,
                               options='MODERATE')



1.14.1


Since this is a hyporthetical model, we will generate ground surface from some some equations

In [59]:
# Note that defining the ground surface this way allows to rediscretize on the fly if needed.
def topElev_Center(x,y):
    return ((-0.003 * x) + 260.) + (((-2E-9 * (x - 5000.)) + 1E-5) * (y + 1500.)**2)

# Model domain and grid definition
Lx = 10000.
Ly = 3000.
nlay = 1
nrow = 30
ncol = 100

delr = Lx / ncol
delc = Ly / nrow

xmax = ncol * delr
ymax = nrow * delc

X, Y = np.meshgrid(np.linspace(delr / 2, xmax - delr / 2, ncol),
                   np.linspace(ymax - delc / 2, 0 + delc / 2, nrow))

ibound = np.ones((nlay, nrow, ncol))

# modify the y values a bit
Y_mm = -1* np.flipud(Y)
topElev = topElev_Center(X, Y_mm)
botElev = np.zeros(topElev.shape)
strtElev= topElev - 1.

Steady = [True,False,False]
nstp = [1, 100, 200]
tsmult = [1, 1, 1]
perlen = [1, 36525, 73050]
    
# Create the discretization object
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, nper=3, delr=delr, delc=delc,
                               top=topElev, botm=botElev, laycbd=0, itmuni=4, lenuni=1,
                               steady=Steady, nstp=nstp, tsmult=tsmult, perlen=perlen)

Now instatiate the UPW package

In [60]:
laytyp = 1
layavg = 0
chani = 1.0
layvka = 0
hdry = -2e+20
iphdry = 1
hk = 0.1
hani = 1
vka = 0.01
ss = 0.00001
sy = 0.20

upw = flopy.modflow.ModflowUpw(mf, laytyp=laytyp, layavg=layavg, chani=chani, layvka=layvka,
                               ipakcb=53, hdry=hdry, iphdry=iphdry, hk=hk, hani=hani,
                               vka=vka, ss=ss, sy=sy)

# Instantiate BAS
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, hnoflo=hdry, strt=strtElev)

Now work on instantiating the boundary conditions

In [61]:
# Recharge Package: The total volume coming in from the upper and lower rows should 
# total 2000.0 on each side (4,000 total)
rech = np.zeros_like(topElev)
individual_cell_rate = 2000. / ncol / (delr*delc)
rech[0,:] = rech[-1,:] = individual_cell_rate
irch = np.zeros_like(rech)  # Remember, 0-based silliness in use
   
rch = flopy.modflow.ModflowRch(mf, nrchop=1, ipakcb=53, rech=rech, irch=irch)

In [62]:
# Evapotranspiration Package: The ET rate is 0.003 everywhere in the model, so rediscretization
# doesn't matter, can keep a constant rate.
nevtop = 1
evtr = 0.003
surf = topElev
exdp = 15
evt = flopy.modflow.ModflowEvt(mf, nevtop=nevtop, ipakcb=53, surf=surf, evtr=evtr, exdp=exdp)

In [89]:
# Streamflow Routing Package: Try and set up with minimal options in use
# 9 11 IFACE # Data Set 1: ISTCB1  ISTCB2
nstrm = 100
nss = 1
const = 1.283833650957E+005
dleak = 0.0001
istcb1 = 53
istcb2 = 0
isfropt = 0
nstrail = 10
isuzn = 1
nsfrsets = 30
irtflg = 1
numtim = 2
weight = 0.75
flwtol = 0.0001
segment_data = None
channel_geometry_data = None
channel_flow_data = None
dataset_5 = None
reachinput = False

# The next couple of lines set up the reach_data for the 30x100 hypothetical model.
# Will need to adjust the row based on which grid discretization we're doing.
# Ensure that the stream goes down one of the middle rows of the model.

# Determine the middle row and store in rMid (account for 0-base)
rMid = nrow//2 - 1

s1 = "k,i,j,iseg,ireach,rchlen,iface\n"
for y in range(ncol):
    #    layer +    row    +        col   +  iseg +   irch      +     rchlen     +    iface
    s1 += '0,' + str(rMid) + ',' + str(y) + ',1,' + str(y+1) + ',' + str(delr) + ',' + '0\n'

s1 = StringIO(s1)
reach_data = np.genfromtxt(s1, delimiter=',',names=True, dtype=[('k', '<f8'), ('i', '<f8'), ('j', '<f8'), ('iseg', '<f8'), ('ireach', '<f8'), ('rchlen', '<f8')])

# Will need to adjust the elevations of the upper and lower ends of the segment based on topElev array
# 5 m below ground surface (5 m of stream incision)
s2 = StringIO(u"nseg,icalc,outseg,iupseg,nstrpts,flow,runoff,etsw,pptsw,roughch,roughbk,cdpth,fdpth,awdth,bwdth,hcond1,thickm1,elevup,width1,depth1,hcond2,thickm2,elevdn,width2,depth2\n1,1,0,0,0,40000.0,0.0,0.0,0.0,0.028,0.0,0.0,0.0,0.0,0.0,1.0,1.0,254.899750,20.0,0.0,1.0,1.0,225.150250,20.0,0.0")
segment_data = np.genfromtxt(s2, delimiter=',',names=True)

# Adjust the elevations of the upstream and downstream ends of the segment
segment_data['elevup'] = topElev[rMid,0] - 5.
segment_data['elevdn'] = topElev[rMid,ncol-1] - 5.

# Be sure to convert segment_data to a dictionary keyed on stress period.
segment_data = np.atleast_1d(segment_data)
segment_data = {0: segment_data,
                1: segment_data,
                2: segment_data}

# There are 2 stress periods
dataset_5 = {0: [nss, 0, 0],
             1: [nss, 0, 0],
             2: [nss, 0, 0]}

channel_flow_data = {0: {1:[[ 1.0E+00, 1.0E+00, 2.5489975E+02, 2.0E+01],    #Data set 6b: HCOND1 THICKM1 ELEVUP WIDTH1
                            [ 1.0E+00, 1.0E+00, 2.2515025E+02, 2.0E+01]]},  #Data set 6c: HCOND2 THICKM2 ELEVDN WIDTH2
                     1: {2:[[ 1.0E+00, 1.0E+00, 2.5489975E+02, 2.0E+01],    #Data set 6b: HCOND1 THICKM1 ELEVUP WIDTH1
                            [ 1.0E+00, 1.0E+00, 2.2515025E+02, 2.0E+01]]},} #Data set 6c: HCOND2 THICKM2 ELEVDN WIDTH2

sfr = flopy.modflow.ModflowSfr2(mf, nstrm=nstrm, nss=nss, const=const, dleak=dleak,
                                ipakcb=53, istcb2=0, reach_data=reach_data, dataset_5=dataset_5,
                                segment_data=segment_data, channel_flow_data=channel_flow_data)

****Warning -- two packages of the same type:  <class 'flopy.modflow.mfsfr2.ModflowSfr2'> <class 'flopy.modflow.mfsfr2.ModflowSfr2'>
replacing existing Package...


In [90]:
# Add a municipal pumping supply well
wel = flopy.modflow.ModflowWel(mf, ipakcb=53, stress_period_data = {1:[0, 11, 74, -1.5e5]})

****Warning -- two packages of the same type:  <class 'flopy.modflow.mfwel.ModflowWel'> <class 'flopy.modflow.mfwel.ModflowWel'>
replacing existing Package...


In [91]:
# Add the Link MT3D package for generating the linker file
lmt = flopy.modflow.ModflowLmt(mf, output_file_name='Spill.ftl', output_file_header='extended',
                               output_file_format='formatted', package_flows = ['sfr'])

****Warning -- two packages of the same type:  <class 'flopy.modflow.mflmt.ModflowLmt'> <class 'flopy.modflow.mflmt.ModflowLmt'>
replacing existing Package...


In [93]:
mf.write_input()
mf.run_model()

FloPy is using the following  executable to run the model: c:\bin\mfnwt.exe

                                  MODFLOW-NWT-SWR1 
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUNDWATER-FLOW MODEL
                             WITH NEWTON FORMULATION
                             Version 1.1.3, 8/01/2017                        
                    BASED ON MODFLOW-2005 Version 1.11.0 08/08/2013                       

                    SWR1 Version 1.04.0 09/15/2016                       

 Using NAME file: Spill.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2018/10/23  7:04:39

 Solving:  Stress period:     1    Time step:     1    Groundwater-Flow Eqn.
 Solving:  Stress period:     2    Time step:     1    Groundwater-Flow Eqn.
 Solving:  Stress period:     2    Time step:     2    Groundwater-Flow Eqn.
 Solving:  Stress period:     2    Time step:     3    Groundwater-Flow Eqn.
 Solving:  Stress period:     2    Time step:     4    Groundwater-Flow Eqn.
 Solving:  Str

 Solving:  Stress period:     2    Time step:   100    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:     1    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:     2    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:     3    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:     4    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:     5    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:     6    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:     7    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:     8    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:     9    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:    10    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:    11    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:    12    Groundwater-Flow Eqn.

 Solving:  Stress period:     3    Time step:   108    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   109    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   110    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   111    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   112    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   113    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   114    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   115    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   116    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   117    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   118    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   119    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:   120    Groundwater-Flow Eqn.

(True, [])

## Setup Transport Model

In [98]:
mt = flopy.mt3d.Mt3dms(modflowmodel=mf, modelname=modelname, model_ws=modelpth,
                       version='mt3d-usgs', namefile_ext='mtnam', exe_name=mtexe,
                       ftlfilename='Spill.ftl', ftlfree=True)

# BTN package
ncomp   =    1
lunit   =    'm'
sconc   =    0.0
prsity  =    sy
cinact  =   -1.0
thkmin  =    0.000001
nprs    =   -1
nprobs  =   10
nprmas  =   10
dt0     =  100.0
nstp    = nstp    # From DIS setup
mxstrn  = 5000
ttsmult =    1.0
ttsmax  =    0

#obs = [[0, 10, 15]]

btn = flopy.mt3d.Mt3dBtn(mt, MFStyleArr=True, DRYCell=True, lunit=lunit,
                         sconc=sconc, ncomp=ncomp, prsity=prsity, cinact=cinact,
                         thkmin=thkmin, nprs=nprs, nprobs=nprobs, 
                         chkmas=True,nprmas=nprmas, dt0=dt0,  
                         mxstrn=mxstrn, ttsmult=ttsmult, ttsmax=ttsmax)
# ADV package
mixelm =    0
percel =    1.0000
mxpart = 5000
nadvfd =    1      # (1 = Upstream weighting)

adv = flopy.mt3d.Mt3dAdv(mt, mixelm=mixelm, percel=percel, mxpart=mxpart, 
                         nadvfd=nadvfd)

#GCG solver package
mxiter =   1
iter1  = 200
isolve =   3
ncrs   =   0
accl   =   1.000000
cclose =   1.00e-06
iprgcg =   5

gcg = flopy.mt3d.Mt3dGcg(mt, mxiter=mxiter, iter1=iter1, 
                         isolve=isolve, ncrs=ncrs, accl=accl, 
                         cclose=cclose, iprgcg=iprgcg)

# SSM
# ---

# First, get a dictionary that has the SSM itype for each of the boundary types.
itype = flopy.mt3d.Mt3dSsm.itype_dict()


mxss = 2          # maximum number of boundary conditions
ssm_data = {}
#             kss,iss,jss,css,        ITYPE
ssm_data[0] = [(0, 11, 74,   0., itype['WEL']),
               (0, 17, 25,   0., itype['CC'])]          #Concstant Concentration
ssm_data[1] = [(0, 11, 74, 100., itype['WEL']),
               (0, 17, 25,   0., itype['CC'])]
ssm_data[2] = [(0, 11, 74,   0., itype['WEL']),
               (0, 17, 25,   0., itype['CC'])]

ssm = flopy.mt3d.Mt3dSsm(mt, mxss=mxss, stress_period_data=ssm_data)

# SFT
# ---
dispsf = 1
coldsf = 0
seg_len = np.unique(reach_data['iseg'], return_counts=True)
obs_sf = np.cumsum(seg_len[1])
obs_sf = obs_sf.tolist()

print(seg_len)
print('value: ' + str(obs_sf))

# In the first and last stress periods, concentration at the headwater is 0.0
sf_stress_period_data = {0: [0, 0, 0.0],
                         1: [0, 0, 0.0],
                         2: [0, 0, 0.0]}

gage_output = [None, None, 'CrnkNic.sftobs']

sft = flopy.mt3d.Mt3dSft(mt, nsfinit=100, mxsfbc=100, icbcsf=81, ioutobs=82,
                         isfsolv=1, cclosesf=1.0E-6, mxitersf=10, crntsf=1.0, iprtxmd=0,
                         coldsf=coldsf, dispsf=dispsf, nobssf=1, obs_sf=obs_sf,
                         sf_stress_period_data = sf_stress_period_data,
                         filenames=gage_output)



found 'rch' in modflow model, resetting crch to 0.0
found 'ets'/'evt' in modflow model, resetting cevt to 0.0
(array([1.]), array([100], dtype=int64))
value: [100]


### Run MT3D-USGS

In [99]:
mt.write_input()
mt.run_model()

FloPy is using the following  executable to run the model: c:\bin\mt3dusgs.exe

 MT3D-USGS - Modular 3D Multi-Species Transport Model [Ver 1.00.00] 
 and based on MT3DMS. MT3D-USGS developed in cooperation by 
 S.S. Papadopulos & Associates and the U.S. Geological Survey

 Using NAME File: Spill.mtnam

 STRESS PERIOD NO.    1

 TIME STEP NO.    1
 FROM TIME =   0.0000     TO    1.0000    

 Transport Step:    1   Step Size:   1.000     Total Elapsed Time:   1.0000    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1

 STRESS PERIOD NO.    2

 TIME STEP NO.    1
 FROM TIME =   1.0000     TO    366.25    

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:   101.00    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:   201.00    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   

 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1

 TIME STEP NO.   18
 FROM TIME =   6210.2     TO    6575.5    

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:   6310.2    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:   6410.2    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   100.0     Total Elapsed Time:   6510.2    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    4   Step Size:   65.25     Total Elapsed Time:   6575.5    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1

 TIME STEP NO.   19
 FROM TIME =   6575.5     TO    6940.8    

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:   6675.5    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1 

 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:   11889.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   100.0     Total Elapsed Time:   11989.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    4   Step Size:   65.25     Total Elapsed Time:   12054.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1

 TIME STEP NO.   34
 FROM TIME =   12054.     TO    12420.    

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:   12154.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:   12254.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   100.0     Total El

 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:   17368.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   100.0     Total Elapsed Time:   17468.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    4   Step Size:   65.25     Total Elapsed Time:   17533.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1

 TIME STEP NO.   49
 FROM TIME =   17533.     TO    17898.    

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:   17633.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:   17733.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   100.0     Total Elapsed Time:   17833.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000    

 Transport Step:    4   Step Size:   65.25     Total Elapsed Time:   22646.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1

 TIME STEP NO.   63
 FROM TIME =   22646.     TO    23012.    

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:   22746.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:   22846.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   100.0     Total Elapsed Time:   22946.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    4   Step Size:   65.25     Total Elapsed Time:   23012.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1

 TIME STEP NO.   64
 FROM TIME =   23012.     TO    23377.    

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:   2

...

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:   91779.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:   91879.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   100.0     Total Elapsed Time:   91979.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    4   Step Size:   65.25     Total Elapsed Time:   92044.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1

 TIME STEP NO.  153
 FROM TIME =   92044.     TO    92409.    

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:   92144.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:   92244.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000    


 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:   97258.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:   97358.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   100.0     Total Elapsed Time:   97458.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    4   Step Size:   65.25     Total Elapsed Time:   97523.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1

 TIME STEP NO.  168
 FROM TIME =   97523.     TO    97888.    

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:   97623.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:   97723.    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000   


 TIME STEP NO.  182
 FROM TIME =  0.10264E+06 TO   0.10300E+06

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:  1.02736E+05
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:  1.02836E+05
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   100.0     Total Elapsed Time:  1.02936E+05
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    4   Step Size:   65.25     Total Elapsed Time:  1.03002E+05
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1

 TIME STEP NO.  183
 FROM TIME =  0.10300E+06 TO   0.10337E+06

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:  1.03102E+05
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:  1.

 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:  1.08315E+05
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   100.0     Total Elapsed Time:  1.08415E+05
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    4   Step Size:   65.25     Total Elapsed Time:  1.08480E+05
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1

 TIME STEP NO.  198
 FROM TIME =  0.10848E+06 TO   0.10885E+06

 Transport Step:    1   Step Size:   100.0     Total Elapsed Time:  1.08580E+05
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   100.0     Total Elapsed Time:  1.08680E+05
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   100.0     Total Elapsed Time:  1.08780E+05
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000    

(False, [])